# Je commencepar importer les librairies dont j'aurais besoin pour la suite du travail

In [1]:
import pandas as pd
import numpy as np
import re

# Importation du fichier

In [2]:
df = pd.read_csv('GSAF5.csv', sep=',',engine='python', encoding='latin1')
#df

In [3]:
df.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN


# Lecture des infos

Je commence par lire les infos macro du fichier

In [4]:
df.shape

(5992, 24)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5992 entries, 0 to 5991
Data columns (total 24 columns):
Case Number               5992 non-null object
Date                      5992 non-null object
Year                      5992 non-null int64
Type                      5992 non-null object
Country                   5949 non-null object
Area                      5590 non-null object
Location                  5496 non-null object
Activity                  5465 non-null object
Name                      5792 non-null object
Sex                       5425 non-null object
Age                       3311 non-null object
Injury                    5965 non-null object
Fatal (Y/N)               5973 non-null object
Time                      2779 non-null object
Species                   3058 non-null object
Investigator or Source    5977 non-null object
pdf                       5992 non-null object
href formula              5991 non-null object
href                      5989 non-null object
C

# Inspection des colonnes
je fais une premiere inspection "grossière" et peu lisible des colonnes

In [6]:
for col in df.columns:
    print(df[col].value_counts())

1980.07.00        2
2009.12.18        2
1920.00.00.b      2
1915.07.06.a.R    2
1990.05.10        2
2005.04.06        2
1923.00.00.a      2
1983.06.15        2
1907.10.16.R      2
2006.09.02        2
2013.10.05        2
1966.12.26        2
1962.06.11.b      2
2012.09.02.b      2
2014.08.02        2
1913.08.27.R      2
1966.05.16.b      1
2006.02.13        1
1890.00.00.b      1
1968.12.29        1
1922.01.04        1
1982.08.00        1
2000.09.00.b      1
1947.03.09        1
1966.06.04        1
1945.00.00.d      1
1888.12.25.R      1
1965.12.10        1
1964.03.00        1
2003.03.10        1
                 ..
1903.03.12        1
1980.11.18        1
1983.10.13        1
1981.11.09        1
2006.05.27        1
1776.00.00.R      1
1967.09.13        1
1988.05.04        1
1954.08.12.b      1
1957.00.00.b      1
1937.10.27.b      1
1961.04.25        1
2016.09.18.c      1
1949.00.00.f      1
1968.08.11        1
1924.02.08        1
1932.08.30.a      1
1994.00.00.b      1
2000.06.30        1


# les premiers points de cleaning à faire:
1.passer les données en dataframe<br>
2.inspecter les données<br>
3.renommer les colonnes pour une meilleures lecture<br>
4.1 renomer la colonne "Sex " en supprimant l'espace --> OK<br>
5.donner le bon type aux données (int, float, date, string, boolean)<br>
6.on peut fusionner dans la colonne Type les données "boat" et "boating" -->OK<br>
7.dans la colonne Country : mettre tous les pays en Majuscule, corriger le nom "ST. MAARTIN" en "ST. MARTIN",<br> 
   supprimer la ligne "Between PORTUGAL & INDIA" et trouver d'autres info du genre--> OK<br>
8.on peut supprimer les carctères spéciaux (les accents)<br>
9.pour la colonne 'Sex', regarger pourquoi il y a 2 M, pour les 3 autres on les remplace par du non défini. --> OK<br>
10.pour les ages, on peut créér une colonne qui sont des tranches d'age<br>
11.pour 'Fatal', nettoyer les différents 'N' ajouter ce qui n'est pas connus par 'UNKNOWN' et 'F' par 'Y'-->OK<br>
12.supprimer une des colonnes "href" car doublon. Si espace dans l'url ajouter un '-'<br>
13.supprimer les doublons en dernier quand le fichier sera la plus clean à mon goût<br>

# point de vigilance
En voulant travailler sur la colonne "Sex", ça indiquait "undefined" --> il y a un problème de syntaxe. il faut donc renommer la colonne pour supprimer l'espace en trop.<br>
pareil pour la colonne "Species"

In [7]:
df =df.rename(columns={'Sex ':'Sex'})
df =df.rename(columns={'Species ':'Species'})
df.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

# Travail sur la colonne 'Sex'

In [8]:
df['Sex'].value_counts() 

M      4835
F       585
M         2
.         1
lli       1
N         1
Name: Sex, dtype: int64

pour la colonne 'Sex', je corrige pour conserver 3 infos : M, F ou UNKNOWN<br/>
Remarque = possibilité de faire un dico pour y mettre mes modif désirées

In [9]:
df['Sex'] =df['Sex'].replace(to_replace =["lli", "N", "."], value ="UNKNOWN") 
df['Sex'] =df['Sex'].replace(to_replace =["M "], value ="M")

"""
--- Autre solution ---
df.loc[df['Sex'] == 'lli']
df.at[1400,'Sex']='UNKNOWN'
df.loc[df['Sex'] == 'N']
df.at[4708,'Sex']='UNKNOWN'
df.loc[df['Sex'] == '.']
df.at[5201,'Sex']='UNKNOWN'
df.loc[df['Sex'] == 'M ']
df.at[341,'Sex']='M'
df.at[1363,'Sex']='M
'"""
#pour Vérification ==> df.loc[df['Sex'] == 'UNKNOWN']
df['Sex'].value_counts() 


M          4837
F           585
UNKNOWN       3
Name: Sex, dtype: int64

# Travail sur la colonne 'Fatal (Y/N)'

In [10]:
df['Fatal (Y/N)'].value_counts() 

N          4315
Y          1552
UNKNOWN      94
 N            8
N             1
n             1
#VALUE!       1
F             1
Name: Fatal (Y/N), dtype: int64

Je choisis de ne garder que 3 infos pour la colonne 'Fatal (Y/N)'<br/>
pour cela je nettois les elements plus haut

In [11]:
df['Fatal (Y/N)'] =df['Fatal (Y/N)'].replace(to_replace =[" N", "n", "N "], value ="N") 
df['Fatal (Y/N)'] =df['Fatal (Y/N)'].replace(to_replace =["F"], value ="Y")
df['Fatal (Y/N)'] =df['Fatal (Y/N)'].replace(to_replace =["#VALUE!"], value ="UNKNOWN")
df['Fatal (Y/N)'].value_counts() 

N          4325
Y          1553
UNKNOWN      95
Name: Fatal (Y/N), dtype: int64

# Travail sur les colonnes 'Year' et 'Date'
Je commence par regarder coment se présentent les 2 colonnes avec value_counts()

In [12]:
df['Year'].value_counts()

2015    139
2011    128
2014    125
0       124
2013    122
2008    121
2009    120
2012    117
2007    112
2006    103
2016    103
2005    103
2010    101
2000     97
1959     93
1960     93
2001     92
2004     92
2003     92
2002     88
1962     86
1961     78
1995     76
1964     66
1998     65
1999     65
1996     61
1963     61
1966     58
1997     57
       ... 
1785      1
1834      1
1791      1
1733      1
1721      1
1637      1
1617      1
77        1
5         1
1703      1
1755      1
1767      1
1771      1
1779      1
1787      1
1803      1
1749      1
1807      1
1811      1
1819      1
1805      1
1831      1
1555      1
1738      1
1859      1
1742      1
1758      1
1818      1
1822      1
1595      1
Name: Year, Length: 232, dtype: int64

il y a 124 annees avec 0 pour infos et 1 champs avec la valeur 5 et 1 autre avec la valeur 77<br/>

In [13]:
df['Date'].value_counts()

1957                        11
1942                         9
1956                         8
1941                         7
1950                         7
1958                         7
1949                         6
No date                      6
28-Jul-95                    5
Aug-56                       5
Oct-60                       5
1940                         5
1955                         5
No date, Before 1963         5
12-Apr-01                    5
05-Oct-03                    5
1954                         5
1970s                        5
1959                         5
28-Dec-14                    4
1945                         4
09-Jul-94                    4
1960                         4
1898                         4
1876                         4
Reported 10-Oct-1906         4
27-Jul-52                    4
23-Jan-70                    4
Before 1958                  4
1961                         4
                            ..
27-Jul-60                    1
04-Oct-7

Dans la colonne 'Date', je constate qu'il y a des cellules qui n'ont pas le format date et qui commencent par un "string". 
Je fais 2 travaux en parallèle : <br/>
1) Je cherche les lignes avec les champs 0 dans la colonne 'Year' et pour les renseigner (si nécessaire) en me servant de la colonne Date<br/>
2) je choisis de remplacer ce qui n'est pas en format date par l'info "No date"<br/>
Pour cela, je crée une df temporaire où je split en 3 colonnes les infos de la colonne 'Date' et je renomme les colonnes en 'jour', 'mois','Annee'<br/>
Je concatene ce nouveau tableau avec la colonne 'Year' de la df d'origine<br/>

In [14]:
df_date= df['Date'].str.split('\W',n=3,expand =True)
df_date.rename(columns={0:'Jour', 1:'Mois', 2:'Annee', 3:'Autres'},inplace=True)

result = pd.concat([df_date, df['Year']], axis=1, join='inner')
result.head()

,Jour,Mois,Annee,Autres,Year
0,18,Sep,16,None,2016
1,18,Sep,16,None,2016
2,18,Sep,16,None,2016
3,17,Sep,16,None,2016
4,16,Sep,16,None,2016


J'identifie les lignes où Year = 0 et où Jour n'est pas un nombre

In [15]:
result_list = result.loc[((result['Year']==0) & (result['Jour'].str.contains('^\D')))]
result_list

,Jour,Mois,Annee,Autres,Year
5868,Ca,,214,B.C.,0
5869,Ca,,336,B.C..,0
5871,Ca,,725,B.C.,0
5873,Before,2016,None,None,0
5874,Before,Oct,2009,None,0
5875,Before,1934,None,None,0
5876,Before,1934,None,None,0
5878,Before,1930,None,None,0
5880,Before,1909,None,None,0
5881,Before,2012,None,None,0


In [16]:
result_list['Jour'].value_counts()

Before     62
No         26
Ca          3
Between     2
World       2
Circa       1
A           1
Beforer     1
Said        1
Early       1
Name: Jour, dtype: int64

Je créé une liste des index pour ensuite remplacer les données qui ne sont pas des dates 

In [17]:
rl = result_list['Jour'].index.to_list()

#rl

Je choisi d'ajouter une colonne nommée 'Date_clean', pour conserver les dates nettoyées.<br/>
    Si j'ai une date je conserve la date<br/>
    Sinon je mets "No date" pour la liste d'index identifiés plus haut

In [18]:
df = df.assign(Date_clean="")
df['Date_clean']=df['Date']

In [19]:
for r in rl:
    df.at[r,'Date_clean']='No date'

In [20]:
df['Date_clean'].value_counts()

No date                     100
1957                         11
1942                          9
1956                          8
1941                          7
1950                          7
1958                          7
1949                          6
1959                          5
1954                          5
05-Oct-03                     5
1940                          5
28-Jul-95                     5
1970s                         5
Oct-60                        5
12-Apr-01                     5
Aug-56                        5
1955                          5
14-Jun-12                     4
09-Jul-94                     4
1952                          4
1898                          4
27-Jul-52                     4
1890                          4
1876                          4
1960                          4
23-Jan-70                     4
1938                          4
28-Dec-14                     4
1960s                         4
                           ... 
12-Oct-8

Dans la df temporaire 'result', j'ai aussi vu des info sous le format 1845-1853<br/>
Pour ces formats, je change aussi les cellules de la colonne 'Date_clean' par "No date"

In [21]:
result_list1 = result.loc[((result['Year']==0) & ((result['Jour'].str.contains('^\d{4}')) & (result['Mois'].str.contains('^\d{4}'))))]
result_list1

,Jour,Mois,Annee,Autres,Year
5879,1880,1899,None,None,0
5893,1896,1913,None,None,0
5949,1941,1945,None,None,0
5953,1941,1945,None,None,0
5954,1941,1945,None,None,0
5955,1941,1942,None,None,0
5959,1940,1946,None,None,0
5989,1900,1905,None,None,0
5990,1883,1889,None,None,0
5991,1845,1853,None,None,0


In [22]:
rl1=result_list1['Jour'].index.to_list()
for r1 in rl1:
    df.at[r1,'Date_clean']='No date'
df['Date_clean'].value_counts()

No date                     110
1957                         11
1942                          9
1956                          8
1958                          7
1950                          7
1941                          7
1949                          6
05-Oct-03                     5
1940                          5
1955                          5
Aug-56                        5
1959                          5
Oct-60                        5
28-Jul-95                     5
12-Apr-01                     5
1954                          5
1970s                         5
23-Jan-70                     4
1960                          4
1961                          4
1876                          4
1960s                         4
09-Jul-94                     4
09-Jan-10                     4
1952                          4
1890                          4
27-Jul-52                     4
1938                          4
1898                          4
                           ... 
12-Oct-8

Je regarde maintenant s'il y a d'autres modifications à réaliser

In [23]:
result_list2 = result.loc[((result['Year']==0) & (result['Jour'].str.contains('\w')==False) & (result['Mois'].str.contains('\D')))]
result_list2

,Jour,Mois,Annee,Autres,Year
5950,,During,the,"war"" 1943-1945",0
5951,,Before,the,"war""",0


In [24]:
#rl2=[5950,5951,5865]
rl2=result_list2['Jour'].index.to_list()
for r2 in rl2:
    df.at[r2,'Date_clean']='No date'
df['Date_clean'].value_counts()

No date                     112
1957                         11
1942                          9
1956                          8
1941                          7
1950                          7
1958                          7
1949                          6
1940                          5
Aug-56                        5
1955                          5
Oct-60                        5
05-Oct-03                     5
1959                          5
1954                          5
1970s                         5
28-Jul-95                     5
12-Apr-01                     5
1945                          4
1960s                         4
1876                          4
1960                          4
23-Jan-70                     4
20-Sep-15                     4
1961                          4
1898                          4
09-Jul-94                     4
1938                          4
1952                          4
1890                          4
                           ... 
11-Nov-0

Pour toutes ces modifications, je n'ai pas eu besoin de toucher à la colonne 'Year'<br/>
Je prends maintenant les cas où la colonne 'Year' est renseignée 

In [25]:
result_list3 = result.loc[(result['Year']!=0) & (result['Jour'].str.contains('Reported')==True) & (result['Autres'].str.contains('\d')==True)]
result_list3.head()

,Jour,Mois,Annee,Autres,Year
33,Reported,,14,Jul-2016,2016
34,Reported,08,Jul,2016,2016
85,Reported,03,Mar,2016,2016
90,Reported,10,Feb,2016,2016
100,Reported,11,Jan,2016,2016


Je supprime le mot 'Reported' des cellules pour ne conserver que la date

In [26]:
df['Date_clean'] = df['Date_clean'].str.lstrip('Reported')
df['Date_clean']

0        18-Sep-16
1        18-Sep-16
2        18-Sep-16
3        17-Sep-16
4        16-Sep-16
5        15-Sep-16
6        11-Sep-16
7        07-Sep-16
8        06-Sep-16
9        05-Sep-16
10       05-Sep-16
11       04-Sep-16
12       01-Sep-16
13       29-Aug-16
14       29-Aug-16
15       27-Aug-16
16       25-Aug-16
17       07-Aug-16
18       06-Aug-16
19       04-Aug-16
20       29-Jul-16
21       28-Jul-16
22       28-Jul-16
23       27-Jul-16
24       26-Jul-16
25       24-Jul-16
26       23-Jul-16
27       23-Jul-16
28       20-Jul-16
29       17-Jul-16
           ...    
5962       No date
5963       No date
5964       No date
5965       No date
5966       No date
5967       No date
5968       No date
5969       No date
5970       No date
5971       No date
5972    1920 -1923
5973       No date
5974       No date
5975       No date
5976       No date
5977       No date
5978       No date
5979       No date
5980       No date
5981       No date
5982       No date
5983       N

In [27]:
df['Date_clean'] = df['Date_clean'].str.lstrip()
[df['Date_clean'].loc[df['Date_clean'].str.contains('happened')]]

[1431    19-Jul-2004 Reported to have happened  "on the...
 5535        14-Sep-1883 (probably happened Ca. 1843/1844)
 Name: Date_clean, dtype: object]

En faisant un value_counts sur la colonne Date_clean, j'ai repéré quelques points à modifier<br/>
#rl2 mettre no date =[5950,5951,5865,5866,5867,4088,5341,5357,5670,1868]
#mettre 'Year' à 0 et 'Date_clean' à No date=[5865,5866,5867]
#retirer le point et la lettre à la fin = [1133,1134,2990]
#réécrire les dates pour = [5776,5794,3883]

In [28]:
result_list4 = result.loc[(result['Year']!=0) & (df['Date'].str.contains('happened')==True) & (result['Autres'].str.contains('\d')==True)]
result_list4

,Jour,Mois,Annee,Autres,Year
1431,,19,Jul,"2004 Reported to have happened ""on the weekend""",2004
5535,Reported,14,Sep,1883 (probably happened Ca. 1843/1844),1883


In [29]:
df.at[[5950,5951,5865,5866,5867,4088,5341,5357,5670,1868,5855,3184,3801,5323,5626,5662,5972],'Date_clean']='No date'
df.at[[5865,5866,5867,5535],'Date_clean']='No date'
df.at[[5865,5866,5867],'Year']=0
df.at[[1133,1134],'Year']='2007'
df.at[[1133,1134],'Date_clean']='19-Jul-2007'
df.at[[2990],'Year']='1975'
df.at[[2990],'Date_clean']='26-Jul-1975'
df.at[[5776],'Date_clean']='16-Jul-1844'
df.at[[5794],'Date_clean']='26-Jul-1836'
df.at[[3883],'Date_clean']='02-Feb-1959'
df.at[1431,'Date_clean']='19 Jul 2004'

In [30]:
 df['Date_clean'].value_counts()

No date           128
1957               11
1942                9
1956                8
1941                7
1950                7
1958                7
1949                6
28-Jul-95           5
12-Apr-01           5
1954                5
Oct-60              5
1938                5
1940                5
05-Oct-03           5
Aug-56              5
1970s               5
1959                5
1898                5
1955                5
1960                4
20-Sep-15           4
1960s               4
27-Dec-08           4
1929                4
1952                4
27-Jul-52           4
1995                4
14-Jun-12           4
1876                4
                 ... 
10-Sep-1959         1
12-Oct-85           1
03-Mar-12           1
19-Dec-89           1
27-Mar-2010         1
25-Nov-62           1
07-Jul-37           1
11-Oct-02           1
Jun-41              1
26-Jan-60           1
1980                1
11-Jul-28           1
Summer of 1883      1
10-Dec-48           1
20-Jun-06 

Je décide de mettre 'No date' pour les dates au format 1xxxs

In [31]:
df['Date_clean'] = df['Date_clean'].str.lstrip()
df['Date_clean'].loc[df['Date'].str.contains('\d\d\d\ds')].value_counts()

No date    8
1970s      5
1960s      4
1700s      3
1950s      2
1980s      2
1820s      1
1920s      1
Name: Date_clean, dtype: int64

In [32]:
df.at[[5852,5853,5854,5816,5028,2855,2857,3183,3187,3188,3189,3190,3802,5931,5932,5933,4264,5904],'Date_clean']='No date'

In [33]:
"""
    #Ancienne méthode
    
pat=[r'Reported\s\d{2}-\w{3}-\d{4}',r'Reported  16-Nov-2005',r'Reported      23-Aug-1998',r'Before\s\w{3}\s\d{4}',r'Before\s\d{4}',r'Circa\s\d{4}',r'Between?',
     r'No date, No date',r'No date No date',r'No date\W\w*',r'No date\W\d*',r'No date\s\w*',r'No date\s\W\w*',
     r'No date\s\W\d*',r'Early\s\w+',r'No date\s\w+',r'No date and\s\w+',r'\d{4}-\d{4}|\d{4}\s-\d{4}',r'\d{4}s','World War II',
     'Ca.\s\d+\sB.C.','Ca.\s\d+.B.C\W+','\d{3}\sB.C.']
for x in pat:
    df['Date_clean'] =df['Date_clean'].replace(regex =x, value ="No date")

#df['Date_clean'][df['Date_clean'].value_counts()].head()
df[df['Date_clean'].str.match('No date')==True]"""

df['Date_clean'].value_counts()

No date           146
1957               11
1942                9
1956                8
1941                7
1958                7
1950                7
1949                6
1959                5
1954                5
28-Jul-95           5
12-Apr-01           5
05-Oct-03           5
Aug-56              5
1955                5
1938                5
1898                5
1940                5
Oct-60              5
20-Sep-15           4
27-Jul-52           4
1890                4
1961                4
1929                4
14-Jun-12           4
1876                4
1952                4
23-Jan-70           4
09-Jul-94           4
1995                4
                 ... 
11-Nov-02           1
10-Sep-1959         1
12-Oct-85           1
25-Jul-11           1
Summer of 1883      1
19-Dec-89           1
25-Nov-62           1
07-Jul-37           1
11-Oct-02           1
Jun-41              1
26-Jan-60           1
27-Mar-2010         1
07-Dec-83           1
17-Jul-00           1
30-Jul-91 

In [34]:
result_list5 = result.loc[(result['Year']!=0) & (df['Date_clean'].str.contains('^\w{3}-')) ]
result_list5.head()

,Jour,Mois,Annee,Autres,Year
151,Sep,15,None,None,2015
282,Sep,14,None,None,2014
303,Aug,14,None,None,2014
496,Dec,12,None,None,2012
579,Apr,13,None,None,2012


In [35]:
rl5=result_list5['Jour'].index.to_list()
for r5 in rl5:
    df.at[r5,'Date_clean']='No date'
df['Date_clean'].value_counts()

No date                                  465
1957                                      11
1942                                       9
1956                                       8
1950                                       7
1958                                       7
1941                                       7
1949                                       6
12-Apr-01                                  5
1938                                       5
1959                                       5
05-Oct-03                                  5
28-Jul-95                                  5
1940                                       5
1954                                       5
1898                                       5
1955                                       5
1961                                       4
1945                                       4
20-Sep-15                                  4
09-Jan-10                                  4
09-Jul-94                                  4
1929      

In [36]:
result_list6 = result.loc[(result['Year']!=0) & (df['Date_clean'].str.contains('^\d{4}')) ]
result_list6.head()

,Jour,Mois,Annee,Autres,Year
366,2014,None,None,None,2014
1485,2004,None,None,None,2004
1854,2000,None,None,None,2000
1918,1999,None,None,None,1999
1919,1999,None,None,None,1999


In [37]:
rl6=result_list6['Jour'].index.to_list()
for r6 in rl6:
    df.at[r6,'Date_clean']='No date'
df['Date_clean'].value_counts()

No date           768
12-Apr-01           5
28-Jul-95           5
05-Oct-03           5
09-Jul-94           4
27-Jul-52           4
23-Jan-70           4
10-Oct-1906         4
28-Dec-14           4
20-Sep-15           4
14-Jun-12           4
27-Dec-08           4
09-Jan-10           4
11-Jan-62           3
27-Jul-04           3
06-Apr-05           3
17-Aug-11           3
19-Aug-07           3
01-Apr-34           3
12-Jul-16           3
24-Jun-15           3
18-May-10           3
14-Sep-03           3
18-Mar-14           3
27-Aug-14           3
07-Jul-16           3
26-Jun-15           3
1940 - 1950         3
25-Jun-13           3
14-May-14           3
                 ... 
27-Jan-08           1
28-Mar-05           1
08-Aug-1907         1
23-Apr-89           1
24-Aug-1928         1
07-Feb-08           1
11-Feb-84           1
15-May-86           1
26-Dec-80           1
14-May-1880         1
15-Mar-12           1
27-Jun-10           1
11-Oct-02           1
07-Jul-37           1
25-Nov-62 

J'applique les dernières modifications

In [38]:
df['Date_clean'].loc[(df['Date_clean'].str.contains('^\d{4}')) | (df['Date'].str.contains('\w or \w')) | (df['Date'].str.contains('^19\d+\s-\s\d+'))]

2628             Mid Jul-1985 or mid Jul-1986
3300                        13 or 30-May-1967
4261                                  No date
4290                                  No date
4937               07-Mar-1925 or 27-Mar-1925
5739                          Sep or Oct-1853
5741                                  No date
5750    "Anniversary Day" 22-Jan-1850 or 1852
5872                             1990 or 1991
5877                                    2009?
5906                                    2003?
5956                              1940 - 1950
5957                              1940 - 1950
5958                              1940 - 1950
Name: Date_clean, dtype: object

In [39]:
df.at[[2628,3300,4261,4290,4937,5739,5741,5750,5872,5877,5906,5956,5957,5958],'Date_clean']='No date'
df['Date_clean'].value_counts()

No date           779
12-Apr-01           5
05-Oct-03           5
28-Jul-95           5
27-Jul-52           4
28-Dec-14           4
09-Jul-94           4
20-Sep-15           4
27-Dec-08           4
09-Jan-10           4
23-Jan-70           4
14-Jun-12           4
10-Oct-1906         4
06-Jan-61           3
26-Jun-15           3
08-Aug-1899         3
18-May-10           3
09-Jun-14           3
07-Jul-16           3
20-Apr-03           3
25-Jun-13           3
07-Jul-12           3
19-Aug-01           3
17-Oct-15           3
14-May-14           3
18-Mar-14           3
28-Jul-13           3
28-Dec-61           3
27-Sep-02           3
11-Jun-62           3
                 ... 
27-Jan-08           1
28-Mar-05           1
08-Aug-1907         1
23-Apr-89           1
24-Aug-1928         1
07-Feb-08           1
11-Feb-84           1
15-May-86           1
26-Dec-80           1
14-May-1880         1
15-Mar-12           1
27-Jun-10           1
11-Oct-02           1
07-Jul-37           1
25-Nov-62 

In [40]:
"""result_list7 = (df.loc[df['Date_clean']=='No date']).index.to_list()

for r7 in result_list7:
    df.at[r7,'Date_clean']='0'
df['Date_clean']"""



"result_list7 = (df.loc[df['Date_clean']=='No date']).index.to_list()\n\nfor r7 in result_list7:\n    df.at[r7,'Date_clean']='0'\ndf['Date_clean']"

In [41]:
"""from datetime import datetime

# Complete the call to convert the date column
df['Date_clean'].value_counts()
df['Date_clean'] = pd.to_datetime(df['Date_clean'],errors='ignore')
df.info()"""


"from datetime import datetime\n\n# Complete the call to convert the date column\ndf['Date_clean'].value_counts()\ndf['Date_clean'] = pd.to_datetime(df['Date_clean'],errors='ignore')\ndf.info()"

# Travail sur la colonne "Type"

In [42]:
df[df['Date'].str.match(r'No date')]

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23,Date_clean
5899,ND.0110,"No date, late 1960s",0,Unprovoked,VENEZUELA,Los Roques Islands,NaN,Spearfishing,4 French divers,M,...,http://waterco.com.br/ataque_tubarao.htm,ND-0110-FrenchDivers.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0110,ND.0110,94,NaN,NaN,No date
5905,ND.0102,"No date, Before 1963",0,Unprovoked,BAHREIN,NaN,NaN,Pearl diving,male,M,...,A.C. Doyle,ND-0102-Bahrein.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0102,ND.0102,88,NaN,NaN,No date
5907,ND.0097,No date,0,Unprovoked,USA,Florida,"Key West, Monroe County",Kitesurfing,Paul Menta,M,...,Internet,ND-0097-PaulMenta.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0097,ND.0097,86,NaN,NaN,No date
5908,ND.0096,No date,0,Unprovoked,REUNION,Grand'Anse,Petite-île,yachtsman in a zodiac,NaN,M,...,G. Van Grevelynghe,ND-0096-Zodiac-Reunion.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0096,ND.0096,85,NaN,NaN,No date
5910,ND.0094,"No date, Before May-1996",0,Unprovoked,KOREA,South Korea,Cheju Island,Diving,"female, a Hae Nyeo",F,...,"K. Amsler, Divernet.com",ND-0094-HaeNyeo.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0094,ND.0094,83,NaN,NaN,No date
5911,ND.0093,"No date, Before Mar-1995",0,Unprovoked,FRENCH POLYNESIA,Tuamotus,Rangiroa,Fishing,male,M,...,J. Windh,ND-0093-Rangiroa.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0093,ND.0093,82,NaN,NaN,No date
5913,ND.0090,"No date, Before Aug-1989",0,Unprovoked,VANUATU,Malampa Province,Malakula,NaN,female,F,...,S. Combs,ND-0090-Vanuatu-female.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0090,ND.0090,80,NaN,NaN,No date
5914,ND.0089,"No date, Before Aug-1987",0,Provoked,VANUATU,Malampa Province,"Hokai, Malakula",Attempting to drive shark from area,a chief,M,...,S. Combs,ND-0089-VanuatuChief.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0089,ND.0089,79,NaN,NaN,No date
5915,ND.0088,"No date, Before 1987",0,Unprovoked,IRAN,Khuzestan Province,"Ahvaz, on the Karun River",NaN,Mr. Jabar-Kaaby,M,...,B. Coad & F. Papahn,ND-0088-Jabar-Kaaby.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0088,ND.0088,78,NaN,NaN,No date
5916,ND.0087,"No date, Before 1975",0,Provoked,USA,Florida,"Riviera Beach, Palm Beach County",Skin diving. Grabbed shark's tail; shark turne...,Carl Bruster,M,...,"R. Skocik, p.176",ND-0087-Carl-Bruster.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0087,ND.0087,77,NaN,NaN,No date


In [43]:
df['Type'].value_counts()

Unprovoked      4386
Provoked         557
Invalid          519
Sea Disaster     220
Boat             200
Boating          110
Name: Type, dtype: int64

pour la colonne 'Type', je fusionne Boat et Boating

In [44]:
df['Type'] =df['Type'].replace(to_replace =["Boating"], value ="Boat")
df['Type'].value_counts()

Unprovoked      4386
Provoked         557
Invalid          519
Boat             310
Sea Disaster     220
Name: Type, dtype: int64

# Travail sur la colonne 'Country'
    #mettre tous les pays en Majuscule,<br/> 
    corriger le nom "ST. MAARTIN" en "ST. MARTIN",<br/>
    #remplacer les NaN par "NC"<br/>
    #passer en "NC" les lignes "Between PORTUGAL & INDIA" ou avec un "?" ou avec "/"<br/>
    #supprimer les espaces avant un mot<br/>

In [45]:
(df['Country'].value_counts()).head(20)

USA                 2116
AUSTRALIA           1279
SOUTH AFRICA         565
PAPUA NEW GUINEA     133
NEW ZEALAND          125
BRAZIL               102
BAHAMAS               98
MEXICO                81
ITALY                 71
FIJI                  62
PHILIPPINES           59
REUNION               57
NEW CALEDONIA         51
MOZAMBIQUE            44
CUBA                  42
SPAIN                 40
INDIA                 37
EGYPT                 36
CROATIA               34
JAPAN                 32
Name: Country, dtype: int64

In [46]:
#passer toutes les données en majuscules
df['Country'] = df['Country'].str.upper()
#identification des NaN et remplacement des NaN par NC (Non Communiqué) car il est difficile après de faire des 'replace'
    #df['Country'].isna()
    #df.loc[df['Country'].isna()]
df['Country']=df['Country'].fillna(value='NC')

df['Country'] =df['Country'].replace(to_replace =["ST. MAARTIN"], value ="ST. MARTIN")

#supprimer le ? de trop
df['Country'] = df['Country'].str.replace('[?]','')
    #df['Country']
    #df[df['Country'].str.match('NC')].head()

#remplacer les emplacements incertains par "NC" (qui ont "beetween", "/" dans l'intitulé)
    #df.loc[df['Country'] == 'NC']
pat5 = df.loc[df['Country'].str.contains(r'( / )')]
df['Country'] =df['Country'].replace(regex =[pat5], value ="NC") 

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


In [47]:
df['Country'].str.lstrip()
#supprimer l'espace au début 
#pat5 = df.loc[df['Country'].str.contains(r'(^/s)')]

df.columns = df.columns.str.lstrip()
(df['Country'].value_counts()).head(20)

USA                 2116
AUSTRALIA           1279
SOUTH AFRICA         565
PAPUA NEW GUINEA     133
NEW ZEALAND          125
BRAZIL               102
BAHAMAS               98
MEXICO                81
ITALY                 71
FIJI                  65
PHILIPPINES           59
REUNION               57
NEW CALEDONIA         51
NC                    50
MOZAMBIQUE            44
CUBA                  42
SPAIN                 40
INDIA                 37
EGYPT                 36
CROATIA               34
Name: Country, dtype: int64

In [48]:
pat6 = df.loc[df['Country'].str.contains(r'(^ )')]
#pat6
df['Country'] =df['Country'].replace(regex =[pat6], value ="")

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """Entry point for launching an IPython kernel.


# Vérificaton des infos dans les colonnes 'Unnamed: 22' & 'Unnamed: 23'

In [49]:
df['Unnamed: 22'].value_counts()

stopped here    1
Name: Unnamed: 22, dtype: int64

In [50]:
df['Unnamed: 23'].value_counts()

change filename    1
Teramo             1
Name: Unnamed: 23, dtype: int64

Comme il n'y a pas d'infos nécessaires, je les supprime

In [51]:
df=df.drop(columns=['Unnamed: 22','Unnamed: 23'],axis=1)

# Travail sur la colonne "pdf"
    Je remplace 'pf' par 'pdf'

In [52]:
pat7 = df.loc[df['pdf'].str.contains(r'pf+')]
df['pdf'] =df['pdf'].replace(regex =[pat7], value ="pdf")
df.loc[df['pdf'].str.contains(r'pdf+')]

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Date_clean
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,13h00,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,18-Sep-16
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,11h00,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,18-Sep-16
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,10h43,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,18-Sep-16
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,17-Sep-16
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,NaN,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,16-Sep-16
5,2016.09.15.R,15-Sep-16,2016,Boat,AUSTRALIA,Western Australia,Bunbury,Fishing,Occupant: Ben Stratton,NaN,...,NaN,NaN,"West Australian, 9/15/2016",2016.09.15.R-boat.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.15.R,2016.09.15.R,5988,15-Sep-16
6,2016.09.11,11-Sep-16,2016,Unprovoked,USA,Florida,"Ponte Vedra, St. Johns County",Wading,male,M,...,15h15,3' to 4' shark,"News4Jax, 9/11/2016",2016.09.11-PonteVedra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.11,2016.09.11,5987,11-Sep-16
7,2016.09.07,07-Sep-16,2016,Unprovoked,USA,Hawaii,"Makaha, Oahu",Swimming,female,F,...,14h30,"Tiger shark, 10?","Hawaii News Now, 9/7/2016",2016.09.07-Oahu.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.07,2016.09.07,5986,07-Sep-16
8,2016.09.06,06-Sep-16,2016,Unprovoked,NEW CALEDONIA,North Province,Koumac,Kite surfing,David Jewell,M,...,15h40,NaN,"TVANouvelles, 9/6/2016",2016.09.06-Jewell.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.06,2016.09.06,5985,06-Sep-16
9,2016.09.05.b,05-Sep-16,2016,Unprovoked,USA,South Carolina,"Kingston Plantation, Myrtle Beach, Horry County",Boogie boarding,Rylie Williams,F,...,Late afternoon,NaN,"C. Creswell, GSAF",2016.09.05.b-Williams.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.05.b,2016.09.05.b,5984,05-Sep-16


# Travail sur la colonne "href"
Identification des NaN et remplacement des NaN par NC (Non Communiqué) car il est difficile après de faire des 'replace'

In [53]:
df['href'].isna()
df.loc[df['href'].isna()]
df['href']=df['href'].fillna(value='NC')
    #remplacer 'pf' par 'pdf'
pat8 = df.loc[df['href'].str.contains(r'pf+')]
df['href'] =df['href'].replace(regex =[pat8], value ="pdf")
df.loc[df['href'].str.contains(r'pdf+')]

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Date_clean
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,13h00,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,18-Sep-16
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,11h00,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,18-Sep-16
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,10h43,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,18-Sep-16
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,17-Sep-16
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,NaN,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,16-Sep-16
5,2016.09.15.R,15-Sep-16,2016,Boat,AUSTRALIA,Western Australia,Bunbury,Fishing,Occupant: Ben Stratton,NaN,...,NaN,NaN,"West Australian, 9/15/2016",2016.09.15.R-boat.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.15.R,2016.09.15.R,5988,15-Sep-16
6,2016.09.11,11-Sep-16,2016,Unprovoked,USA,Florida,"Ponte Vedra, St. Johns County",Wading,male,M,...,15h15,3' to 4' shark,"News4Jax, 9/11/2016",2016.09.11-PonteVedra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.11,2016.09.11,5987,11-Sep-16
7,2016.09.07,07-Sep-16,2016,Unprovoked,USA,Hawaii,"Makaha, Oahu",Swimming,female,F,...,14h30,"Tiger shark, 10?","Hawaii News Now, 9/7/2016",2016.09.07-Oahu.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.07,2016.09.07,5986,07-Sep-16
8,2016.09.06,06-Sep-16,2016,Unprovoked,NEW CALEDONIA,North Province,Koumac,Kite surfing,David Jewell,M,...,15h40,NaN,"TVANouvelles, 9/6/2016",2016.09.06-Jewell.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.06,2016.09.06,5985,06-Sep-16
9,2016.09.05.b,05-Sep-16,2016,Unprovoked,USA,South Carolina,"Kingston Plantation, Myrtle Beach, Horry County",Boogie boarding,Rylie Williams,F,...,Late afternoon,NaN,"C. Creswell, GSAF",2016.09.05.b-Williams.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.05.b,2016.09.05.b,5984,05-Sep-16


# Suppression de la colonne "href formula" 
car identique à la colonne "href"

In [54]:
df = df.drop(columns=['href formula'],axis=1)

# Travail sur la colonne "Time"
    remplacer les NaN

In [55]:
df['Time'].value_counts()
df.loc[df['Time'].isna()]
df['Time']=df['Time'].fillna(value='NC')

Je change la syntaxe des élements ayant une heure correcte mais mal écrite pour la suite du nettoyage.<br/>
En effet, avec ce format de date je vais pouvoir remplir la colonne "AM_PM" créée et ensuite les changer en AM, PM ou NC.
        #A faire plus tard  --> df['Time']= df['Time'].str.replace('h',':',regex=True)

In [56]:
df['Time'] =df['Time'].replace(to_replace =["06j00"], value ="06h00")
df['Time'] =df['Time'].replace(to_replace =["8:04 PM"], value ="20h04")
df['Time'] =df['Time'].replace(to_replace =["15j45"], value ="15h45")
df['Time'] =df['Time'].replace(to_replace =["1300"], value ="13h00")
df['Time'] =df['Time'].replace(to_replace =["1600"], value ="16h00")
df['Time'] =df['Time'].replace(to_replace =["1500"], value ="15h00")
df['Time'] =df['Time'].replace(to_replace =["13h345"], value ="13h34")

### Creation d'un nouvelle colonne "AM_PM" pour y reseigner les infos suivantes : le matin-AM de l'apres-midi-PM si pas possible ou vide alors NC


In [57]:
df = df.assign(AM_PM="")
df['AM_PM']=df['Time']

df[['Time','AM_PM']].head(15)

,Time,AM_PM
0,13h00,13h00
1,11h00,11h00
2,10h43,10h43
3,NC,NC
4,NC,NC
5,NC,NC
6,15h15,15h15
7,14h30,14h30
8,15h40,15h40
9,Late afternoon,Late afternoon


In [58]:
df['AM_PM'].sort_values(ascending=False)

1631                                        
3588                                   night
5645                                    dusk
2485                                       X
1149                                  Sunset
1664                                  Sunset
1593                                  Sunset
1214                                  Sunset
250           Sometime between 06h00 & 08hoo
1718                    Shortly before 13h00
1619                  Shortly after midnight
4488                Ship aban-doned at 03h10
2767                          Prior to 10h37
1801    Possibly same incident as 2000.08.21
1952                                    P.M.
3752                                    P.M.
3795                                    P.M.
4332                                    P.M.
3923                                    P.M.
4005                                    P.M.
4188                                    P.M.
1842                                    P.M.
1673      

Retour au travail sur la colonne "AM_PM"<br/>
    Je remplace toutes les heures en info AM ou PM suivant le format

In [59]:
pat9=(df['AM_PM'].str.contains(r'[0-1][0-2]h[0-5][0-9]'))
pat10=(df['AM_PM'].str.contains(r'([1]?[2-9]|2[0-3])h[0-5][0-9]'))

df['AM_PM'] = df['Time']
mask1 = (pat9==True)
df.loc[mask1, 'AM_PM'] = 'AM'


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


In [60]:
mask2 = (pat10==True)
df.loc[mask2, 'AM_PM'] = 'PM'
df['AM_PM'].value_counts()

NC                                                                       3213
PM                                                                       1822
AM                                                                        422
Afternoon                                                                 174
Morning                                                                   107
Night                                                                      58
Evening                                                                    33
Late afternoon                                                             32
Dusk                                                                       15
A.M.                                                                       12
P.M.                                                                       12
Early morning                                                              11
Midday                                                          

En checkant les valeurs uniques, on constate qu'il y a une info qui devrait renseigner la colonne "Fatal (Y/N)"<br/>
    Je cherche donc l'index 

In [61]:
df.loc[df['Time'].str.contains('FATAL')]    
    #df.at[4776].value_counts()
    #je cherche ensuite si la colonne suivante n'a pas l'info
    #pour cela, j'ai réduit le tableau pour vérifier s'il n'y a pas de décalage 
df1 = df[['Sex','Age','Injury','Fatal (Y/N)','Time','Species','Investigator or Source','pdf']]
df1.loc[4776]
#mais en fait c'est ok. Donc je ne corrige que la colonne "Fatal" pour cet index
df.at[4776,'Fatal (Y/N)']="Y"

In [62]:
pat11 = r'^\s$|^\s\s$|^\s\s\s$|Before daybreak|"shortly PM|After 04h00|Daytime|500|830|2 hours after Opperman|--|FATAL\s\s(\w+.)+|30 minutes after 1992.07.08.a|Possibly same incident as 2000.08.21|Sometime between 06h00 & 08hoo|>08h00|>06h45|<07h30|10h30 or 13h30|10h00 or 14h00|X'
pat12 = r'"Early evening"|Shortly after midnight|Nightfall|After midnight|A.M.|[A-Za-z]+\s[Aa]fterno*n|PM.|Midday.|"PM|"Night"|"After lunch"|Noon|[Aa]fterno*n|Lunchtime|Just before sundown|[Aa]fter\sno*n|Late night|[Mm]idnight|18h15-18h30|[Nn]ight+|[Ee]vening|evening+|"Evening"|[A-Za-z]+\s[Dd]usk|[Dd]usk|Dark|"After dark"|[Ss]unset|Midday|17h00 or 17h40|Just after 12h00|"shortly before dusk"' 
pat13 = r'P.M.|[A-Za-z]+\s[Mm]orning|Mid-morning|[Mm]orning|[Dd]awn|Just before dawn|[A-Za-z]+\s[Dd]awn|[Aa]fter\smidnight|Just before noon|"Just before 11h00"|Just AM|AM\s|Daybreak|2 hrs before PM|07h00 - 08h00|10h45-11h15|09h30 / 10h00|Prior to 10h37|11h00 / 11h30|09h00 - 09h30|09h00 -10h00|Before 07h00|10h00 -- 11h00|06h00 -- 07h00|Between 05h00 & 08h00|Between 06h00 & 07h20|10h00 / 11h00|03h45 / 04h00|08h00 / 09h30|Before 10h30|Between 11h00 & 12h00|After 04h00|11h01 -time of ship sinking|Ship aban-doned at 03h10' 
df['AM_PM'] =df['AM_PM'].replace(regex =[pat11], value ="NC") 
df['AM_PM'] =df['AM_PM'].replace(regex =[pat12], value ="PM")
df['AM_PM'] =df['AM_PM'].replace(regex =[pat13], value ="AM")
df.at[62,'AM_PM']='AM'

In [63]:
df[['Time','AM_PM']].head(15)

,Time,AM_PM
0,13h00,PM
1,11h00,AM
2,10h43,AM
3,NC,NC
4,NC,NC
5,NC,NC
6,15h15,PM
7,14h30,PM
8,15h40,PM
9,Late afternoon,PM


In [64]:
df['AM_PM'].value_counts()

NC                                                                       3231
PM                                                                       2196
AM                                                                        564
FATAL  (Wire netting installed at local beaches after this incident.)       1
Name: AM_PM, dtype: int64

In [65]:
df.loc[df['Time'].str.contains(r'FATAL+')]

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Time,Species,Investigator or Source,pdf,href,Case Number.1,Case Number.2,original order,Date_clean,AM_PM
4776,1931.08.31,31-Aug-31,1931,Unprovoked,CUBA,Havana Province,"Vedado Baths, Miramar, Havana",Swimming,Manuel Romero,M,...,FATAL (Wire netting installed at local beache...,NaN,"NYTimes, 9/2/1931",1931.08.31-Romero.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,1931.08.31,1931.08.31,1217,31-Aug-31,FATAL (Wire netting installed at local beache...


In [66]:
df.at[4776,'AM_PM']='AM'

In [67]:
df['AM_PM'].value_counts()

NC    3231
PM    2196
AM     565
Name: AM_PM, dtype: int64

In [68]:
df['Time']= df['Time'].str.replace('h',':',regex=True)
df['Time']

0                13:00
1                11:00
2                10:43
3                   NC
4                   NC
5                   NC
6                15:15
7                14:30
8                15:40
9       Late afternoon
10      Late afternoon
11                  NC
12                  NC
13               15:00
14               14:00
15               17:00
16               16:00
17                  NC
18                  NC
19                  NC
20               11:30
21                  NC
22                  NC
23                  NC
24               12:00
25               19:05
26                  NC
27                  NC
28          After noon
29                  NC
             ...      
5962                NC
5963                NC
5964                NC
5965                NC
5966                NC
5967                NC
5968                NC
5969                NC
5970                NC
5971                NC
5972                NC
5973                NC
5974       

# Exportation d'un fichier csv propre

In [69]:
df.to_csv('shark_cleaning.csv', sep=',', index=False)